## В этом ноутбуке мы подготавливаем базовые аггрегаты фичей для нашей модели

Также нужно присвоить в переменную PATH_TO_RFM путь до файлика из ноутбука RFM.ipynb

Также нужно присвоить в переменную PATH_TO_TRAIN путь до файлика трейновой выборки

Также нужно присвоить в переменную PATH_TO_TEST путь до файлика тестовой выборки

In [72]:
PATH_TO_RFM = "RFM_Segment.csv"
PATH_TO_TRAIN = "train_dataset_hackaton2023_train.gzip"
PATH_TO_TEST = "test_dataset_hackaton2023_test.gzip"

In [1]:
import pandas as pd

In [ ]:
df_train = pd.read_parquet('/home/pedashenko/Загрузки/train_dataset_hackaton2023_train.gzip')
df_test = pd.read_parquet('/home/pedashenko/Загрузки/test_dataset_hackaton2023_test.gzip')
df_all = pd.concat([df_train, df_test])
df_train_true = pd.read_parquet('/home/pedashenko/Загрузки/train_dataset_hackaton2023_train.gzip')
df_test_true = pd.read_parquet('/home/pedashenko/Загрузки/test_dataset_hackaton2023_test.gzip')

In [3]:
del df_train
del df_test

In [5]:
df_train = df_all

In [18]:
df_check_amount = df_train.groupby(['customer_id', 'startdatetime']).agg({'revenue': ['sum']}).reset_index()

In [7]:
df_simple_agg = df_check_amount.groupby(['customer_id']).agg({
#    ('revenue', 'sum'): ['mean', 'sum', 'max', 'min', 'std', 'nunique', scipy.stats.kurtosis, scipy.stats.skew, 'count']
    ('revenue', 'sum'): ['mean', 'sum', 'max', 'min', 'std', 'nunique', 'count']
})
df_simple_agg = df_simple_agg.reset_index()


In [8]:
def prepare_columns(df_aggs):
    df_aggs.columns = ['_'.join(col) if type(col) is tuple else col for col in df_aggs.columns]
    df_aggs = df_aggs.rename(columns={'customer_id__': 'customer_id'})
    
    return df_aggs


def get_target(df_aggs):
    assert 'customer_id' in df_aggs.columns 
    df_target = df_train.groupby('customer_id').agg({'buy_post': ['max']}).reset_index()
    df_target.columns = ['customer_id', 'buy_post']
    df_joined = df_aggs.set_index("customer_id").join(df_target.set_index('customer_id'))
    df_joined = df_joined.reset_index()
#    df_joined.columns = ['_'.join(col) if type(col) is tuple else col for col in df_joined.columns]
#    df_joined = df_joined.rename(columns={'customer_id__': 'customer_id'})
    
    return df_joined

In [9]:
df_agg_prepared = prepare_columns(df_simple_agg)
df_agg_final = get_target(df_agg_prepared)

In [10]:
df_check_amount = df_train.groupby(['customer_id', 'startdatetime']).agg({'revenue': ['sum']}).reset_index()
df_check_amount.columns = ['customer_id', 'startdatetime', 'revenue_sum']
df_check_amount = df_check_amount.set_index('customer_id')
df_check_amount['max_startdatetime'] =  df_check_amount.groupby('customer_id').agg({'startdatetime': ['max']})
df_last_check = df_check_amount[df_check_amount['startdatetime'] == df_check_amount['max_startdatetime']]
df_last_check = df_last_check[['revenue_sum']]
df_last_check.columns = ['revenue_last']
df_agg_final_last = df_last_check[['revenue_last']].join(df_agg_final.set_index('customer_id'))
#df_agg_final_last.reset_index().to_csv("train_user_order_aggs_v3.csv", index=False)

In [20]:
#df_check_amount = df_check_amount.reset_index()
df_check_amount.columns = ['customer_id', 'startdatetime', 'revenue_sum']
df_min_time = df_check_amount.groupby('customer_id').agg({'startdatetime': ['min']})
df_check_amount = df_check_amount.set_index("customer_id")
df_min_time.columns = ['startdatetime_min']
df_joined_tmp = df_min_time.join(df_check_amount)

In [22]:
import numpy as np



def get_diff(x):
    return x.diff().dropna().dt.days.tolist()


df_explode = df_joined_tmp.groupby('customer_id').agg({'startdatetime': get_diff})

In [25]:
df_check_amount = df_train.groupby(['customer_id', 'startdatetime']).agg({'revenue': ['sum']}).reset_index()
df_check_amount.columns = ['customer_id', 'startdatetime', 'revenue_sum']
df_check_amount = df_check_amount.set_index('customer_id')
df_check_amount['max_startdatetime'] =  df_check_amount.groupby('customer_id').agg({'startdatetime': ['max']})
df_last_check = df_check_amount[df_check_amount['startdatetime'] == df_check_amount['max_startdatetime']]
df_last_check = df_last_check[['revenue_sum']]
df_last_check.columns = ['revenue_last']
df_agg_final_last = df_last_check[['revenue_last']].join(df_agg_final.set_index('customer_id'))

In [23]:
df_explode2 = df_explode.explode('startdatetime')
df_explode3 = df_explode2.reset_index().groupby('customer_id').agg({'startdatetime': ['mean', 'min', 'max', 'std', 'nunique', 'count']})
df_explode3.columns = ['_'.join(col) if type(col) is tuple else col for col in df_explode3.columns]
df_explode3.columns = [col.replace('startdatetime', 'delta_days_order') if 'startdatetime' in col else col for col in df_explode3.columns]

In [26]:
df_agg_final_last2 = df_explode3.join(df_agg_final_last)

In [27]:
df_check_max_min = df_check_amount.reset_index().groupby(['customer_id']).agg({'startdatetime': ['max', 'min']})
df_range = (df_check_max_min['startdatetime']['max'] - df_check_max_min['startdatetime']['min']).dt.days
df_min_max_range = pd.DataFrame(df_range).rename(columns={0:  'max_min_delta_days' })
df_agg_final_last3 = df_agg_final_last2.join(df_min_max_range)
df_agg_final_last3['LTV'] = df_agg_final_last3['revenue_sum_mean'] * df_agg_final_last3['delta_days_order_mean']

In [28]:
def min_second(x):
    if len(x) >= 2:
        return sorted(x)[1]
    return -1
    
    
def max_second(x):
    if len(x) >= 2:
        return sorted(x)[-1]
    return -1
    
    
def min_third(x):
    if len(x) >= 3:
        return sorted(x)[2]
    return -1
    
    
def max_third(x):
    if len(x) >= 3:
        return sorted(x)[-2]
    return -1



df_agg = df_check_amount.reset_index().groupby('customer_id').agg({
    'startdatetime': [min_second, max_second, min_third, max_third]
})

In [29]:
df_agg.columns = ['_'.join(col) if type(col) is tuple else col for col in df_agg.columns]

In [30]:
df_time = df_check_amount.join(df_agg)

In [35]:
df_time_min_second = df_time[df_time['startdatetime'] == df_time['startdatetime_min_second']][['revenue_sum']]
df_time_min_third = df_time[df_time['startdatetime'] == df_time['startdatetime_min_third']][['revenue_sum']]

df_time_max_second = df_time[df_time['startdatetime'] == df_time['startdatetime_max_second']][['revenue_sum']]
df_time_max_third = df_time[df_time['startdatetime'] == df_time['startdatetime_max_third']][['revenue_sum']]

In [33]:
# df_agg_final_last3.drop('revenue_sum_min_second'.split('\t'), axis=1, inplace=True)

In [37]:
df_time_max_third

,revenue_sum
customer_id,
29891,264.96
30477,229.99
31426,299.99
44491,49.97
44939,554.94
...,...
46632765,209.99
46639170,399.97
46654016,179.99


In [38]:
df_time_min_second = df_time_min_second.rename(columns={'revenue_sum': 'revenue_sum_min_second'})
df_time_min_third = df_time_min_third.rename(columns={'revenue_sum': 'revenue_sum_min_third'})

df_time_max_second = df_time_max_second.rename(columns={'revenue_sum': 'revenue_sum_max_second'})
df_time_max_third = df_time_max_third.rename(columns={'revenue_sum': 'revenue_sum_max_third'})

In [39]:
df_agg_final_last3 = df_agg_final_last3.join(df_time_min_second)
df_agg_final_last3 = df_agg_final_last3.join(df_time_min_third)
df_agg_final_last3 = df_agg_final_last3.join(df_time_max_third)
df_agg_final_last3 = df_agg_final_last3.join(df_time_max_second)
# df_agg_final_last3.to_csv("train_user_order_aggs_v6.csv", index=False)

In [40]:
df_tmp10 = df_train[df_train['revenue'] < 10.0]
df_tmp1 = df_train[df_train['revenue'] == 1.0]
df_tmp2 = df_tmp1.groupby('customer_id').agg({'revenue': ['sum']})
df_tmp2.columns = ['number_ones']
df_tmp10_sum = df_tmp10.groupby('customer_id').agg({'revenue': ['sum', 'count']})
df_tmp10_sum.columns = ['sum_revenue_less_10', 'count_revenue_less_10']
df_agg_final_last3['number_ones'] = df_tmp2
df_agg_final_last3 = df_agg_final_last3.join(df_tmp10_sum)
df_agg_final_last3 = df_agg_final_last3.fillna(0.0)

In [41]:
df_agg_final_last3 = df_agg_final_last3.reset_index()

In [ ]:
## Добавить еще

df_agg_final_last3['month_max'] = df_check_max_min['startdatetime']['max'].dt.month
df_agg_final_last3['month_min'] = df_check_max_min['startdatetime']['min'].dt.month

df_agg_final_last3['day_of_week_max'] = df_check_max_min['startdatetime']['max'].dt.day_of_week
df_agg_final_last3['day_of_week_min'] = df_check_max_min['startdatetime']['min'].dt.day_of_week
df_check_amount['day_of_week'] = df_check_amount['startdatetime'].dt.day_of_week

In [45]:
df_check_amount = df_train.groupby(['customer_id', 'startdatetime']).agg({'revenue': ['sum']}).reset_index()
df_check_amount = df_check_amount.reset_index()
df_check_amount.columns = ['customer_id', 'startdatetime', 'revenue_sum', 'day_of_week']
df_mode = df_check_amount.groupby('customer_id').agg({'day_of_week': [lambda x: x.value_counts().index[0]]})
df_mode.columns = ['days_of_week_mode']
df_agg_final_last3['days_of_week_mode'] = df_mode['days_of_week_mode']
df_agg_final_last3 = df_agg_final_last3.rename(columns={'day_of_week_max': 'day_of_week_last', 'day_of_week_min': 'day_of_week_first'})
df_check_amount['day_of_week'] = df_check_amount['revenue_sum'].dt.day_of_week

In [53]:
df_check_amount

,customer_id,startdatetime,revenue_sum,day_of_week
0,0,29891,2022-12-05 12:03:58,0
1,1,29891,2022-12-05 14:28:35,0
2,2,29891,2022-12-15 00:37:19,3
3,3,29891,2022-12-20 09:20:38,1
4,4,29891,2022-12-21 09:46:23,2
...,...,...,...,...
3472854,3472854,46661804,2023-08-01 18:01:40,1
3472855,3472855,46661804,2023-08-01 18:04:56,1
3472856,3472856,46668221,2023-08-01 20:46:27,1
3472857,3472857,46668221,2023-08-01 20:56:56,1


In [55]:
rfm_df = pd.read_csv(PATH_TO_RFM)

In [56]:
last_day = rfm_df.groupby('customer_id').agg({'date': ['max']})
last_day.columns = ['date_max']
rfm_df = rfm_df.set_index('customer_id').join(last_day)

In [ ]:
rfm_last_day = rfm_df[rfm_df['date_max'] == rfm_df['date']]
rfm_last_day = rfm_last_day.reset_index().drop_duplicates('customer_id')
df_agg_final_last3 = df_agg_final_last3.set_index('customer_id').join(rfm_last_day.set_index('customer_id'))
df_agg_final_last3 = df_agg_final_last3.reset_index()
df_agg_final_last3.columns = [col + '_last' if idx >=63 else col for idx, col in enumerate(df_agg_final_last3.columns)]

In [ ]:
df_agg_final_last3.drop(['date_max_last', 'date_last'], axis=1, inplace=True)

In [60]:
rfm_last_day = rfm_df[rfm_df['date_max'] == rfm_df['date']]
rfm_last_day = rfm_last_day.reset_index().drop_duplicates('customer_id')

In [65]:
rfm_last_day.columns = [col + '_last' if col not in ['customer_id', 'date', 'id_restoran'] else col for col in rfm_last_day.columns.tolist()]

In [69]:
df_agg_final_last3 = df_agg_final_last3.set_index('customer_id').join(rfm_last_day.set_index('customer_id'))

In [81]:
df_agg_final_last3 = df_agg_final_last3.reset_index()

In [84]:

df_agg_final_last3[df_agg_final_last3['customer_id'].isin(df_train['customer_id'].unique())]

,customer_id,delta_days_order_mean,delta_days_order_min,delta_days_order_max,delta_days_order_std,delta_days_order_nunique,delta_days_order_count,revenue_last,revenue_sum_mean,revenue_sum_sum,...,date,id_restoran,median_last,sum_last,count_last,R_Quartile_last,F_Quartile_last,M_Quartile_last,RFMClass_last,date_max_last
0,29891,2.000000,0,11,3.036016,8,24,439.98,203.494000,5087.35,...,2023-02-01,Отдельно стоящий с внешней зоной 463.0,439.98,439.98,1,1,3,1,131,2023-02-01
1,30477,1.583333,0,9,2.552350,8,24,44.99,227.024000,5675.60,...,2022-11-25,Отдельно стоящий без внешней зоны 320.0,44.99,44.99,1,4,1,1,411,2022-11-25
2,31426,1.739130,0,6,1.814522,5,23,44.99,391.399583,9393.59,...,2023-07-01,Фудкорт без туалета 153.0,39.99,344.98,7,4,2,3,423,2023-07-01
3,44491,8.666667,0,26,15.011107,2,3,69.99,128.725000,514.90,...,2023-07-08,Отдельно стоящий без внешней зоны без туалета ...,23.00,69.99,3,4,1,2,412,2023-07-08
4,44939,7.500000,7,8,0.707107,2,2,504.96,554.943333,1664.83,...,2022-12-25,Отдельно стоящий с внешней зоной 280.0,51.04,504.96,7,4,3,3,433,2022-12-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612329,46632765,0.000000,0,0,0.000000,1,2,289.98,246.650000,739.95,...,2023-08-01,Не заполнено 225.0,89.99,289.98,3,3,2,2,322,2023-08-01
612330,46639170,0.000000,0,0,0.000000,1,2,519.96,363.306667,1089.92,...,2023-07-31,Фудкорт без туалета 80.2,119.99,1089.92,11,2,4,4,244,2023-07-31
612331,46654016,0.000000,0,0,0.000000,1,2,569.98,406.643333,1219.93,...,2023-08-01,Отдельно стоящий с внешней зоной 270.0,159.99,1219.93,6,1,4,3,143,2023-08-01
612332,46661804,0.000000,0,0,0.000000,1,2,289.97,278.310000,834.93,...,2023-08-01,Фудкорт с туалетом 300.0,70.03,834.93,15,3,4,4,344,2023-08-01


In [77]:
df_agg_final_last3.reset_index().to_csv("all_aggs.csv", index=False)

In [86]:
df_agg_final_last3[df_agg_final_last3['customer_id'].isin(df_train['customer_id'].unique())].to_csv("train_user_order_aggs_v13.csv", index=False)

True    612334
Name: customer_id, dtype: int64

In [ ]:
df_agg_final_last3[df_agg_final_last3['customer_id'].isin(df_train_true['customer_id'].unique())].to_csv("test_user_order_aggs_v13.csv", index=False)